# Efficient Config Copying with AutoConfig and AutoProcessor

This notebook demonstrates the optimal approach for copying configuration files:
- Use `AutoConfig.from_pretrained(model_id)` directly
- Use `AutoTokenizer.from_pretrained(model_id)` for NLP models
- Use `AutoProcessor.from_pretrained(model_id)` for vision/multimodal models

**Key insight**: We don't need to load the full model weights, just the config files!

## Setup

In [1]:
from transformers import AutoConfig, AutoTokenizer, AutoProcessor, AutoImageProcessor
from pathlib import Path
import json

# Test output directory
output_dir = Path("../models/config-only-test")
output_dir.mkdir(parents=True, exist_ok=True)

## 1. Copy Config Files for BERT (NLP Model)

In [2]:
def copy_config_files(model_id: str, output_dir: Path):
    """Copy config files from HF model ID without loading model weights."""
    
    print(f"📁 Copying configs from '{model_id}'")
    
    # 1. Config (always required)
    config = AutoConfig.from_pretrained(model_id)
    config.save_pretrained(output_dir)
    print("   ✅ config.json")
    
    # 2. Tokenizer (for NLP models)
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        tokenizer.save_pretrained(output_dir)
        print("   ✅ tokenizer files")
    except Exception as e:
        print(f"   ℹ️  No tokenizer (vision/audio model)")
    
    # 3. Image Processor (for vision models)
    try:
        processor = AutoImageProcessor.from_pretrained(model_id)
        processor.save_pretrained(output_dir)
        print("   ✅ image processor")
    except Exception:
        print("   ℹ️  No image processor (NLP model)")
    
    # 4. General Processor (for multimodal)
    try:
        processor = AutoProcessor.from_pretrained(model_id)
        processor.save_pretrained(output_dir)
        print("   ✅ general processor")
    except Exception:
        pass
    
    # Summary
    files = list(output_dir.glob("*"))
    total_size = sum(f.stat().st_size for f in files)
    print(f"\n   📊 {len(files)} files, {total_size/1024:.1f} KB total")
    return files

In [3]:
# Test with BERT
bert_dir = output_dir / "bert-tiny"
bert_dir.mkdir(exist_ok=True)

bert_files = copy_config_files("prajjwal1/bert-tiny", bert_dir)
print(f"\nCreated files: {[f.name for f in bert_files]}")

📁 Copying configs from 'prajjwal1/bert-tiny'
   ✅ config.json
   ✅ tokenizer files
   ℹ️  No image processor (NLP model)
   ✅ general processor

   📊 5 files, 922.7 KB total

Created files: ['config.json', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'vocab.txt']


   ✅ config.json


   ✅ tokenizer files


   ℹ️  No image processor (NLP model)


   ✅ general processor

   📊 5 files, 922.7 KB total

Created files: ['config.json', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'vocab.txt']


## 2. Copy Config Files for Vision Model (ViT)

In [4]:
# Test with Vision Transformer
vit_dir = output_dir / "vit-base"
vit_dir.mkdir(exist_ok=True)

vit_files = copy_config_files("google/vit-base-patch16-224", vit_dir)
print(f"\nCreated files: {[f.name for f in vit_files]}")

📁 Copying configs from 'google/vit-base-patch16-224'
   ✅ config.json
   ℹ️  No tokenizer (vision/audio model)


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


   ✅ image processor


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


   ✅ general processor

   📊 2 files, 68.4 KB total

Created files: ['config.json', 'preprocessor_config.json']


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


   ✅ general processor

   📊 2 files, 68.4 KB total

Created files: ['config.json', 'preprocessor_config.json']


## 3. Copy Config Files for Multimodal Model (CLIP)

In [5]:
# Test with CLIP (multimodal)
clip_dir = output_dir / "clip-base"
clip_dir.mkdir(exist_ok=True)

clip_files = copy_config_files("openai/clip-vit-base-patch32", clip_dir)
print(f"\nCreated files: {[f.name for f in clip_files]}")

📁 Copying configs from 'openai/clip-vit-base-patch32'
   ✅ config.json
   ✅ tokenizer files


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


   ✅ image processor


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


   ✅ general processor

   📊 7 files, 4914.1 KB total

Created files: ['config.json', 'preprocessor_config.json', 'vocab.json', 'special_tokens_map.json', 'merges.txt', 'tokenizer.json', 'tokenizer_config.json']


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


   ✅ general processor

   📊 7 files, 4914.1 KB total

Created files: ['config.json', 'preprocessor_config.json', 'vocab.json', 'special_tokens_map.json', 'merges.txt', 'tokenizer.json', 'tokenizer_config.json']


## 4. Validate Config Contents

In [6]:
# Load and inspect the BERT config
with open(bert_dir / "config.json") as f:
    bert_config = json.load(f)

print("BERT Config highlights:")
print(f"  Architecture: {bert_config.get('architectures', [])}")
print(f"  Model type: {bert_config.get('model_type')}")
print(f"  Hidden size: {bert_config.get('hidden_size')}")
print(f"  Num layers: {bert_config.get('num_hidden_layers')}")

BERT Config highlights:
  Architecture: []
  Model type: bert
  Hidden size: 128
  Num layers: 2


In [7]:
# Load and inspect the ViT config
with open(vit_dir / "config.json") as f:
    vit_config = json.load(f)

print("ViT Config highlights:")
print(f"  Architecture: {vit_config.get('architectures', [])}")
print(f"  Model type: {vit_config.get('model_type')}")
print(f"  Image size: {vit_config.get('image_size')}")
print(f"  Patch size: {vit_config.get('patch_size')}")

ViT Config highlights:
  Architecture: ['ViTForImageClassification']
  Model type: vit
  Image size: 224
  Patch size: 16


## 5. Test with Existing ONNX Model

In [8]:
# Simulate adding configs to existing ONNX export
import shutil

# Create a test directory with mock ONNX file
test_export = output_dir / "simulated-export"
test_export.mkdir(exist_ok=True)

# Create a dummy ONNX file (in real scenario, this comes from HTP export)
dummy_onnx = test_export / "model.onnx"
with open(dummy_onnx, "w") as f:
    f.write("# This would be the actual ONNX model from HTP export\n")

print(f"1. Created mock ONNX export: {test_export}")
print(f"   Files before: {[f.name for f in test_export.glob('*')]}")

1. Created mock ONNX export: ../models/config-only-test/simulated-export
   Files before: ['config.json', 'special_tokens_map.json', 'model.onnx', 'tokenizer.json', 'tokenizer_config.json', 'vocab.txt']


In [9]:
# Add config files to the existing export
print("\n2. Adding config files using model ID:")
copy_config_files("prajjwal1/bert-tiny", test_export)

print(f"\n   Files after: {[f.name for f in test_export.glob('*')]}")


2. Adding config files using model ID:
📁 Copying configs from 'prajjwal1/bert-tiny'
   ✅ config.json
   ✅ tokenizer files
   ℹ️  No image processor (NLP model)
   ✅ general processor

   📊 6 files, 922.7 KB total

   Files after: ['config.json', 'special_tokens_map.json', 'model.onnx', 'tokenizer.json', 'tokenizer_config.json', 'vocab.txt']


   ✅ config.json


   ✅ tokenizer files


   ℹ️  No image processor (NLP model)


   ✅ general processor

   📊 6 files, 922.7 KB total

   Files after: ['config.json', 'special_tokens_map.json', 'model.onnx', 'tokenizer.json', 'tokenizer_config.json', 'vocab.txt']


## 6. Implementation for ModelExport

In [10]:
def export_with_config(model_id: str, output_dir: Path, onnx_filename: str = "model.onnx"):
    """
    Complete export function implementing our "Always Copy Configuration" approach.
    
    This is the function we'll implement in the HTP exporter.
    """
    print(f"🚀 Exporting {model_id} with config files")
    
    # Step 1: Export ONNX (this would be our HTP export)
    # export_onnx_with_hierarchy(model_id, output_dir / onnx_filename)
    print(f"   1. Export ONNX with HTP -> {onnx_filename}")
    
    # Step 2: Copy config files (the focus of this notebook)
    print(f"   2. Copy config files from model ID")
    
    # Config (always required)
    config = AutoConfig.from_pretrained(model_id)
    config.save_pretrained(output_dir)
    
    # Tokenizer (conditional)
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        tokenizer.save_pretrained(output_dir)
        print(f"      ✅ Tokenizer saved")
    except Exception:
        print(f"      ℹ️  No tokenizer (not an NLP model)")
    
    # Image processor (conditional)
    try:
        processor = AutoImageProcessor.from_pretrained(model_id)
        processor.save_pretrained(output_dir)
        print(f"      ✅ Image processor saved")
    except Exception:
        print(f"      ℹ️  No image processor (not a vision model)")
    
    # Step 3: Validate
    print(f"   3. Validate export")
    if not (output_dir / "config.json").exists():
        raise FileNotFoundError("config.json not created - export failed!")
    
    files = list(output_dir.glob("*"))
    print(f"   ✅ Export complete: {len(files)} files")
    
    return output_dir

# Test the complete function
demo_export = output_dir / "complete-export-demo"
demo_export.mkdir(exist_ok=True)

result = export_with_config("prajjwal1/bert-tiny", demo_export)
print(f"\n🎉 Export result: {[f.name for f in result.glob('*')]}")

🚀 Exporting prajjwal1/bert-tiny with config files
   1. Export ONNX with HTP -> model.onnx
   2. Copy config files from model ID
      ✅ Tokenizer saved
      ℹ️  No image processor (not a vision model)
   3. Validate export
   ✅ Export complete: 5 files

🎉 Export result: ['config.json', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'vocab.txt']


      ✅ Tokenizer saved


      ℹ️  No image processor (not a vision model)
   3. Validate export
   ✅ Export complete: 5 files

🎉 Export result: ['config.json', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'vocab.txt']


## 7. Performance Analysis

In [11]:
import time

def benchmark_config_copying():
    """Benchmark how fast config copying is."""
    
    models_to_test = [
        "prajjwal1/bert-tiny",
        "google/vit-base-patch16-224", 
        "openai/clip-vit-base-patch32"
    ]
    
    print("⏱️  Config Copying Performance:")
    print("   Model".ljust(35), "Time (ms)", "Files", "Size (KB)")
    print("   " + "-" * 60)
    
    for model_id in models_to_test:
        test_dir = output_dir / f"benchmark-{model_id.replace('/', '-')}"
        test_dir.mkdir(exist_ok=True)
        
        # Clean directory
        for f in test_dir.glob("*"):
            f.unlink()
        
        # Time the config copying
        start_time = time.time()
        
        config = AutoConfig.from_pretrained(model_id)
        config.save_pretrained(test_dir)
        
        try:
            tokenizer = AutoTokenizer.from_pretrained(model_id)
            tokenizer.save_pretrained(test_dir)
        except:
            pass
        
        try:
            processor = AutoImageProcessor.from_pretrained(model_id)
            processor.save_pretrained(test_dir)
        except:
            pass
        
        elapsed_ms = (time.time() - start_time) * 1000
        
        # Calculate stats
        files = list(test_dir.glob("*"))
        total_size = sum(f.stat().st_size for f in files)
        
        model_short = model_id.split('/')[-1][:20]
        print(f"   {model_short.ljust(35)} {elapsed_ms:>6.1f} {len(files):>5} {total_size/1024:>8.1f}")

benchmark_config_copying()

⏱️  Config Copying Performance:
   Model                            Time (ms) Files Size (KB)
   ------------------------------------------------------------
   bert-tiny                           2763.1     5    922.7


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


   vit-base-patch16-224                1585.5     2     68.4
   clip-vit-base-patch3                1164.4     7   4914.0


   vit-base-patch16-224                1634.7     2     68.4


   clip-vit-base-patch3                1117.3     7   4914.0


## Key Takeaways

### ✅ Benefits of Config-Only Approach

1. **Much Faster**: No need to download/load model weights (GBs)
2. **Memory Efficient**: Only loads config files (KBs)
3. **Universal**: Works for any HuggingFace model
4. **Simple**: Direct API calls, no complex logic

### 📝 Implementation Pattern

```python
def add_configs_to_export(model_id: str, output_dir: Path):
    # Always copy config.json (required by Optimum)
    config = AutoConfig.from_pretrained(model_id)
    config.save_pretrained(output_dir)
    
    # Try tokenizer (for NLP models)
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        tokenizer.save_pretrained(output_dir)
    except: pass
    
    # Try image processor (for vision models) 
    try:
        processor = AutoImageProcessor.from_pretrained(model_id)
        processor.save_pretrained(output_dir)
    except: pass
```

### 🚀 Next Steps

1. Integrate this pattern into HTP exporter
2. Update CLI to use `export_with_config()` by default
3. Add validation tests
4. Create production-ready implementation